In [ ]:
# Дополните класс CustomDecisionTreeClassifier
# Код методов plot_node, plot_tree и count_nodes изменять не нужно

class CustomDecisionTreeClassifier():
    """
    Простой классификатор на основе дерева решений с критерием Джини в качестве критерия разделения.

    Аргументы:frrrrrrrrrrrrr
        max_depth (int): Максимальная глубина дерева. По умолчанию — 3.

    Атрибуты:
        label (int): Метка класса, который наиболее часто встречается в узле.
        feature (str): Признак, используемый для разделения.
        size (int): Количество объектов в узле.
        threshold (float): Пороговое значение для разделения.
        left (CustomDecisionTreeClassifier): Левое поддерево (значение <= порог).
        right (CustomDecisionTreeClassifier): Правое поддерево (значение > порог).
        gini (float): Индекс Джини в узле.
    """
    def __init__(self, max_depth=3):
        self.max_depth = max_depth
        self.label = None
        self.feature = None
        self.size = None
        self.threshold = np.nan
        self.left = None
        self.right = None
        self.gini = 0

    def gini_impurity(self, y):
        """
        Вычисляет индекс Джини для массива меток классов.

        Аргументы:
            y (numpy.ndarray): Массив меток классов.

        Возвращает:
            float: Значение индекса Джини.
        """
        _, counts = np.unique(y, return_counts=True)
        probabilities = counts / len(y)
        return 1 - np.sum(probabilities ** 2)

    def best_split(self, X, y):
        """
        Находит оптимальный признак и порог для разделения данных.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив меток классов.

        Возвращает:
            tuple:
                str: Оптимальный признак для разделения.
                float: Оптимальное пороговое значение.
                float: Значение индекса Джини после оптимального разделения.
        """
        best_feature = None
        best_threshold = None
        best_gini = self.gini_impurity(y)
        N = X.shape[0]
        for feature in X.columns:
            values = np.sort(X[feature].unique())
            thresholds = [(values[i] + values[i+1]) / 2 for i in range(len(values)-1)]
            for threshold in thresholds:
                mask_left = X[feature] <= threshold
                mask_right = ~mask_left
                N_left = mask_left.sum() 
                N_right = mask_right.sum()
                if N_left == 0 or N_right == 0:
                    continue  
                gini_left = self.gini_impurity(y[mask_left])
                gini_right = self.gini_impurity(y[mask_right])
                weighted_gini = (N_left * gini_left + N_right * gini_right) / N
                if weighted_gini < best_gini:
                    best_gini = weighted_gini
                    best_feature = feature
                    best_threshold = threshold

        return best_feature, best_threshold, best_gini

    def fit(self, X, y):
        """
        Обучает дерево решений, рекурсивно находя оптимальные разделения.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив меток классов.

        Возвращает:
            CustomDecisionTreeClassifier: Обученное дерево решений.
        """
        self.size = len(y)
        classes, counts = np.unique(y, return_counts=True)
        self.label = classes[np.argmax(counts)]
        self.gini = self.gini_impurity(y)
        if self.max_depth == 0:
            return 
        self.feature, self.threshold, gini = self.best_split(X, y)
        if self.feature is None or gini >= self.gini:
            return
        self.gini = gini
        mask_left = X[self.feature] <= self.threshold
        mask_right = ~mask_left
        self.left = CustomDecisionTreeClassifier(max_depth=self.max_depth-1)
        self.right = CustomDecisionTreeClassifier(max_depth=self.max_depth-1)
        self.left.fit(X[mask_left], y[mask_left])
        self.right.fit(X[mask_right], y[mask_right])
        return self

    def predict(self, X):
        """
        Предсказывает метки классов.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.

        Возвращает:
            numpy.ndarray: Массив предсказанных меток классов.
        """
        if self.feature is None:
            return np.full(X.shape[0], self.label)  
        mask_left = X[self.feature] <= self.threshold
        mask_right = ~mask_left
        y_pred = np.empty(X.shape[0])
        y_pred[mask_left] = self.left.predict(X[mask_left])
        y_pred[mask_right] = self.right.predict(X[mask_right])
        
        return y_pred

    def plot_node(self, dot, node_id=0):
        """
        Вспомогательный метод для визуализации дерева.

        Аргументы:
            dot (graphviz.Digraph): Объект Digraph библиотеки graphviz.
            node_id (int): Идентификатор текущего узла. По умолчанию — 0.

        Возвращает:
            int: Идентификатор узла.
        """
        if self.feature is None:
            label = f"gini = {self.gini:.3f}\nsamples = {self.size}\nlabel = {self.label}"
            dot.node(str(node_id), label=label, fillcolor="#8ccd96")
            return node_id
        label = f"{self.feature} <= {self.threshold:.3f}\ngini = {self.gini:.3f}\nsamples = {self.size}\nlabel = {self.label}"
        dot.node(str(node_id), label=label, fillcolor="#ffffff")
        left_id = self.left.plot_node(dot, node_id*2 + 1)
        dot.edge(str(node_id), str(left_id))
        right_id = self.right.plot_node(dot, node_id*2 + 2)
        dot.edge(str(node_id), str(right_id))
        
        return node_id

    def plot_tree(self, filled=True):
        """
        Генерирует визуализацию дерева решений.

        Аргументы:
            filled (bool): Закрашивать узлы. По умолчанию — True.

        Возвращает:
            graphviz.Digraph: Объект graphviz с визуализацией дерева.
        """
        dot = graphviz.Digraph()
        dot.attr('node', shape='box', style='filled' if filled else None)
        self.plot_node(dot)
        return dot
    
    def count_nodes(self):
        """
        Рассчитывает сложность дерева — совокупное количество узлов в дереве.

        Возвращает:
            int: Общее количество узлов.
        """
        if self.feature is None:
            return 1
        return 1 + self.left.count_nodes() + self.right.count_nodes()

In [ ]:
# Дополните класс CustomDecisionTreeRegressor
# Код методов plot_node, plot_tree и count_nodes изменять не нужно

class CustomDecisionTreeRegressor():
    """
    Простой регрессор на основе дерева решений с ошибкой MSE в качестве критерия разделения и средним значением в листе для прогноза.

    Аргументы:
        max_depth (int): Максимальная глубина дерева. По умолчанию — 8.

    Атрибуты:
        value (float): Среднее значение целевой переменной в узле.
        feature (str): Признак, используемый для разделения.
        size (int): Количество объектов в узле.
        threshold (float): Пороговое значение для разделения.
        left (CustomDecisionTreeRegressor): Левое поддерево (значение <= порог).
        right (CustomDecisionTreeRegressor): Правое поддерево (значение > порог).
        mse (float): Ошибка MSE в узле.
    """
    def __init__(self, max_depth=8):
        self.max_depth = max_depth
        self.value = None
        self.feature = None
        self.size = None
        self.threshold = np.nan
        self.left = None
        self.right = None
        self.mse = 0


    def MSE(self, y_true, y_pred):
        """
        Вычисляет среднеквадратичную ошибку (MSE).

        Аргументы:
            y_true (numpy.ndarray): Массив истинных значений целевой переменной.
            y_pred (numpy.ndarray): Массив предсказанных значений целевой переменной.

        Возвращает:
            float: Значение MSE.
        """
        return np.mean((y_true - y_pred) ** 2)


    def best_split(self, X, y):
        """
        Находит оптимальный признак и порог для разделения данных.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив значений целевой переменной.

        Возвращает:
            tuple:
                str: Оптимальный признак для разделения.
                float: Оптимальное пороговое значение.
                float: Значение среднеквадратичной ошибки (MSE) после разделения.
        """
        best_feature = None
        best_threshold = None
        best_mse = self.MSE(y, y.mean())
        N = X.shape[0]
        for feature in X.columns:
            values = np.sort(X[feature].unique())
            thresholds = [(values[i] + values[i+1]) / 2 for i in range(len(values) - 1)]
            for threshold in thresholds:
                mask_left = X[feature] <= threshold
                mask_right = ~mask_left
                N_left = mask_left.sum()
                N_right = mask_right.sum()
                if N_left == 0 or N_right == 0:
                    continue
                loss_left = self.MSE(y[mask_left], y[mask_left].mean())
                loss_right = self.MSE(y[mask_right], y[mask_right].mean())
                weighted_mse = (N_left * loss_left + N_right * loss_right) / N
                if weighted_mse < best_mse:
                    best_mse = weighted_mse
                    best_feature = feature
                    best_threshold = threshold
        return best_feature, best_threshold, best_mse


    def fit(self, X, y):
        """
        Обучает дерево решений, рекурсивно находя оптимальные разделения.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив значений целевой переменной.

        Возвращает:
            CustomDecisionTreeRegressor: Обученное дерево решений.
        """
        self.value = y.mean()
        self.mse = self.MSE(y, self.value)
        self.size = len(y)
        if self.max_depth == 0:
            return
        self.feature, self.threshold, mse = self.best_split(X, y)
        if self.feature is None or mse >= self.mse:
            return
        self.mse = mse
        mask_left = X[self.feature] <= self.threshold
        mask_right = ~mask_left
        self.left = CustomDecisionTreeRegressor(max_depth=self.max_depth-1)
        self.right = CustomDecisionTreeRegressor(max_depth=self.max_depth-1)
        self.left.fit(X[mask_left], y[mask_left])
        self.right.fit(X[mask_right], y[mask_right])
        return self

    def predict(self, X):
        """
        Предсказывает значения целевой переменной.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.

        Возвращает:
            numpy.ndarray: Массив предсказанных значений целевой переменной.
        """
        if self.feature is None:
            return np.full(X.shape[0], self.value)
        mask_left = X[self.feature] <= self.threshold
        mask_right = ~mask_left
        y_pred = np.empty(X.shape[0])
        y_pred[mask_left] = self.left.predict(X[mask_left])
        y_pred[mask_right] = self.right.predict(X[mask_right])
        return y_pred
    
    def prune(self, epsilon):
        """
        Обрезает дерево, если разница ошибок между родительским и дочерними узлами меньше epsilon.
        После обрезки внутренний узел становится листом.

        Аргументы:
            epsilon (float): Пороговое значение для обрезки.
        """
        if self.feature is None:
            return
        self.left.prune(epsilon)
        self.right.prune(epsilon)
        left_size = self.left.size
        right_size = self.right.size
        left_mse = self.left.mse
        right_mse = self.right.mse
        weighted_mse = (left_size * left_mse + right_size * right_mse) / (left_size + right_size)
        if weighted_mse > 0 and (self.mse - weighted_mse) < epsilon:
            self.feature = None
            self.threshold = np.nan
            self.left = None
            self.right = None
    
    def plot_node(self, dot, node_id=0):
        """
        Вспомогательный метод для визуализации дерева.

        Аргументы:
            dot (graphviz.Digraph): Объект Digraph библиотеки graphviz.
            node_id (int): Идентификатор текущего узла. По умолчанию — 0.

        Возвращает:
            int: Идентификатор узла.
        """
        if self.feature == None:
            dot.node(str(node_id), label='mse = {:.3f}\nsamples = {}\nvalue = {:.3f}'.format(self.mse, self.size, self.value), fillcolor="#8ccd96")
            return node_id
        dot.node(str(node_id), label='{} <= {:.3f}\nmse = {:.3f}\nsamples = {}\nvalue = {:.3f}'
                 .format(self.feature, self.threshold, self.mse, self.size, self.value), fillcolor="#ffffff")
        left_id = self.left.plot_node(dot, node_id*2 + 1)
        dot.edge(str(node_id), str(left_id))
        right_id = self.right.plot_node(dot, node_id*2 + 2)
        dot.edge(str(node_id), str(right_id))
        return node_id
    
    def plot_tree(self, filled=True):
        """
        Генерирует визуализацию дерева решений.

        Аргументы:
            filled (bool): Закрашивать узлы. По умолчанию — True.

        Возвращает:
            graphviz.Digraph: Объект graphviz с визуализацией дерева.
        """
        dot = graphviz.Digraph()
        dot.attr('node', shape='box', style='filled' if filled else None)
        self.plot_node(dot)
        return dot
    
    def count_nodes(self):
        """
        Рассчитывает сложность дерева — совокупное количество узлов в дереве.

        Возвращает:
            int: Общее количество узлов.
        """
        if self.feature is None:
            return 1
        return 1 + self.left.count_nodes() + self.right.count_nodes()

In [ ]:
# класс CustomGradientBoostingRegressor


class CustomGradientBoostingRegressor(BaseEstimator):
    """
    Простой регрессор на основе градиентного бустинга над деревьями решений.

    Аргументы:
        n_estimators (int): Количество деревьев (итераций бустинга). По умолчанию — 100.
        learning_rate (float) Темп обучения (шаг градиентного спуска). По умолчанию — 0.1.
        max_depth (int): Максимальная глубина дерева бустинга. По умолчанию — 1.
        random_state : (int|None): Сид для фиксирования случайного состояния. По умолчанию — None (не фиксировать).

    Атрибуты:
        f0 (float): Начальное предсказание (среднее значение целевой переменной).
        models (list[DecisionTreeRegressor]): Последовательность деревьев, составляющих модель градиентного бустинга.
    """
    def __init__(self, n_estimators=100, learning_rate=0.1, max_depth=1, random_state=None):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.random_state = random_state
        self.f0 = None
        self.models = []

    def fit(self, X, y):
        """
        Обучает модель градиентного бустинга.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив значений целевой переменной.

        Возвращает: 
            CustomGradientBoostingRegressor: Обученная модель градиентного бустинга.
        """
        self.f0 = np.mean(y)
        y_pred = np.full_like(y, self.f0)
        for _ in range(self.n_estimators):
            residuals = y - y_pred
            tree = DecisionTreeRegressor(max_depth=self.max_depth, random_state=self.random_state)
            tree.fit(X, residuals)
            y_pred += self.learning_rate * tree.predict(X)
            self.models.append(tree)
        return self

    def predict(self, X):
        """
        Предсказывает значения целевой переменной.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.

        Возвращает:
            numpy.ndarray: Массив предсказанных значений целевой переменной.
        """
        y_pred = np.full(X.shape[0], self.f0)
        for model in self.models:
            y_pred += self.learning_rate * model.predict(X)
        return y_pred
    
    def score(self, X, y):
        """
        Вычисляет отрицательное значение MSE (Negative MSE) для прогноза.
        Метод необходим для применения GridSearchCV.

        Аргументы:
            X (pandas.DataFrame): Таблица с признаками.
            y (numpy.ndarray): Массив значений целевой переменной.

        Возвращает:
            float: Значение Negative MSE.
        """
        return -mean_squared_error(y, self.predict(X))